# Set up environment

Don't run the following Cell if you are using local machine

In [ ]:
!git clone https://{GITHUB_ACCESS_TOKEN}@github.com/AliMohseninejad/ganbert-classifier.git
!rm ganbert-classifier/Codes/main.ipynb
!cp -r ganbert-classifier/Codes/data/ ./
!cp -r ganbert-classifier/Codes/evaluation/ ./
!cp -r ganbert-classifier/Codes/model/ ./
!cp -r ganbert-classifier/Codes/training/ ./
!cp -r ganbert-classifier/Dataset/ ../
!cp -r ganbert-classifier/Plots/ ../

In [ ]:
!pip install -qU transformers

Run the following cell only if you are using google colab.
The dataset should be available on your google drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
dataset_path = "drive/MyDrive/"

Run the following cell only if you are using Kaggle. The dataset should be first uploaded to Kaggle as the "subtaskB" dataset.

In [ ]:
dataset_path = "/kaggle/input/subtaskB/"

Run the following cell only if you are using local machine. The dataset should be in the "Dataset" folder.

In [1]:
dataset_path = "../Dataset/"

In [ ]:
import torch
import numpy as np
import os

from data.data_loader import generate_dataloader
from model.bert import get_bert_model, get_tokenizer, get_bert_model_with_adapter
from model.discriminator import Discriminator
from model.generator1 import Generator1
from training.train import train_vanilla_classier, train_gan
from evaluation.test import test_vanilla_bert, test_gan_bert
from training.visualize import plot_results, plot_results_gan

/home/ali/Courses/deep_learning/hw/deep_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Vanilla BERT

In [3]:
torch.manual_seed(42)
train_batch_size = 4
val_test_batch_size = 4
epochs = 10
learning_rate = 5e-5
max_size = 64
model_name = "bert-base-cased"
bert_tokenizer, bert_config = get_tokenizer(model_name=model_name)

In [ ]:
if not os.path.exists("../Plots/vanilla-bert/"):
    os.mkdir(path="../Plots/vanilla-bert/")

In [ ]:
bert_acc_list = []
bert_f1_list = []
for supervised_ratio in [0.01, 0.05, 0.10, 0.50]:
    bert_save_path = f"../Plots/vanilla-bert/bert_{int(100*(supervised_ratio))}sup.pth"
    discriminator_save_path = (
        f"../Plots/vanilla-bert/discriminator_{int(100*(supervised_ratio))}sup.pth"
    )
    plot_save_path = f"../Plots/vanilla-bert/"

    bert_model, _ = get_bert_model(model_name=model_name)
    classifier = Discriminator(num_labels=6)

    # Get dataloaders
    train_dataloader, val_dataloader, test_dataloader = generate_dataloader(
        dataset_folder_path=dataset_path,
        unsupervised_ratio=1-supervised_ratio,
        tokenizer=bert_tokenizer,
        train_batch_size=train_batch_size,
        valid_batch_size=val_test_batch_size,
        test_batch_size=val_test_batch_size,
        use_unsup=False,
        max_length=max_size,
        use_bow_dataset=False,
        random_seed=42,
    )

    # Define optimizer
    model_params = [v for v in bert_model.parameters()] + [
        v for v in classifier.parameters()
    ]
    optimizer = torch.optim.AdamW(model_params, lr=learning_rate)

    # Train the model
    bert_model, classifier, vanilla_training_results = train_vanilla_classier(
        transformer=bert_model,
        classifier=classifier,
        optimizer=optimizer,
        epochs=epochs,
        scheduler=None,
        train_dataloader=train_dataloader,
        validation_dataloader=val_dataloader,
        transformer_path=bert_save_path,
        classifier_path=discriminator_save_path,
    )

    
    # Test the model
    test_acc, test_f1 = test_vanilla_bert(
        bert_model_name=model_name,
        transformer_path=bert_save_path,
        discriminator_path=discriminator_save_path,
        test_dataloader=test_dataloader
    )
    bert_acc_list.append(test_acc)
    bert_f1_list.append(test_f1)
    title_suffix = " " + str(int(100*(supervised_ratio))) + f"% supervised"
    # Visualize results
    plot_results(vanilla_training_results, title_suffix, plot_save_path)

np.save(file="../Plots/vanilla-bert/acc.npy", arr=np.array(bert_acc_list))
np.save(file="../Plots/vanilla-bert/f1.npy", arr=np.array(bert_f1_list))

# Bert + Adapter

In [ ]:
torch.manual_seed(42)
train_batch_size = 4
val_test_batch_size = 4
epochs = 10
learning_rate = 5e-5
max_size = 64
model_name = "bert-base-cased"
bert_tokenizer, bert_config = get_tokenizer(model_name=model_name)

In [ ]:
if not os.path.exists("../Plots/adapter-bert/"):
    os.mkdir(path="../Plots/adapter-bert/")

In [ ]:
adapter_bert_acc_list = []
adapter_bert_f1_list = []
for supervised_ratio in [0.01, 0.05, 0.10, 0.50]:
    bert_save_path = f"../Plots/adapter-bert/bert_{int(100*(supervised_ratio))}sup.pth"
    discriminator_save_path = (
        f"../Plots/adapter-bert/discriminator_{int(100*(supervised_ratio))}sup.pth"
    )
    plot_save_path = f"../Plots/adapter-bert/"

    bert_model, _ = get_bert_model_with_adapter(model_name=model_name)
    classifier = Discriminator(num_labels=6)

    # Get dataloaders
    train_dataloader, val_dataloader, test_dataloader = generate_dataloader(
        dataset_folder_path=dataset_path,
        unsupervised_ratio=1-supervised_ratio,
        tokenizer=bert_tokenizer,
        train_batch_size=train_batch_size,
        valid_batch_size=val_test_batch_size,
        test_batch_size=val_test_batch_size,
        use_unsup=False,
        max_length=max_size,
        use_bow_dataset=False,
        random_seed=42,
    )

    # Define optimizer
    model_params = [v for v in bert_model.parameters()] + [
        v for v in classifier.parameters()
    ]
    optimizer = torch.optim.AdamW(model_params, lr=learning_rate)

    # Train the model
    bert_model, classifier, vanilla_training_results = train_vanilla_classier(
        transformer=bert_model,
        classifier=classifier,
        optimizer=optimizer,
        epochs=epochs,
        scheduler=None,
        train_dataloader=train_dataloader,
        validation_dataloader=val_dataloader,
        transformer_path=bert_save_path,
        classifier_path=discriminator_save_path,
    )

    
    # Test the model
    test_acc, test_f1 = test_vanilla_bert(
        bert_model_name=model_name,
        transformer_path=bert_save_path,
        discriminator_path=discriminator_save_path,
        test_dataloader=test_dataloader
    )

    adapter_bert_acc_list.append(test_acc)
    adapter_bert_f1_list.append(test_f1)
    title_suffix = " " + str(int(100*(supervised_ratio))) + f"% supervised" 
    # Visualize results
    plot_results(vanilla_training_results, title_suffix, plot_save_path)

np.save(file="../Plots/adapter-bert/acc.npy", arr=np.array(adapter_bert_acc_list))
np.save(file="../Plots/adapter-bert/f1.npy", arr=np.array(adapter_bert_f1_list))

# GAN-BERT

## G1

In [ ]:
torch.manual_seed(42)
train_batch_size = 4
val_test_batch_size = 4
epochs = 10
learning_rate_discriminator = 5e-5
learning_rate_generator = 5e-5
max_size = 64
model_name = "bert-base-cased"
bert_tokenizer, bert_config = get_tokenizer(model_name=model_name)

In [ ]:
if not os.path.exists("../Plots/generator1/"):
    os.mkdir(path="../Plots/generator1/")

In [ ]:
g1_bert_acc_list = []
g1_bert_f1_list = []
for unsupervised_ratio in [0.99, 0.95, 0.90, 0.50]:
    bert_save_path = (
        f"../Plots/generator1/bert_{int(100*(1-unsupervised_ratio))}sup.pth"
    )
    discriminator_save_path = (
        f"../Plots/generator1/discriminator_{int(100*(1-unsupervised_ratio))}sup.pth"
    )
    generator_save_path = (
        f"../Plots/generator1/generator_{int(100*(1-unsupervised_ratio))}sup.pth"
    )
    plot_save_path = f"../Plots/generator1/"

    bert_model, _ = get_bert_model(model_name=model_name)
    classifier = Discriminator()
    generator = Generator1()

    # Get dataloaders
    train_dataloader, val_dataloader, test_dataloader = generate_dataloader(
        dataset_folder_path=dataset_path,
        unsupervised_ratio=unsupervised_ratio,
        tokenizer=bert_tokenizer,
        train_batch_size=train_batch_size,
        valid_batch_size=val_test_batch_size,
        test_batch_size=val_test_batch_size,
        max_length=max_size,
        use_unsup=True,
        use_bow_dataset=False,
        random_seed=42,
    )

    # Define optimizers
    discriminator_params = [v for v in bert_model.parameters()] + [
        v for v in classifier.parameters()
    ]
    generator_params = [v for v in generator.parameters()]
    d_optimizer = torch.optim.AdamW(
        discriminator_params, lr=learning_rate_discriminator
    )
    g_optimizer = torch.optim.AdamW(generator_params, lr=learning_rate_generator)

    bert_model, generator, classifier, gan1_training_results = train_gan(
        transformer=bert_model,
        generator=generator,
        discriminator=classifier,
        bow_mode=False,
        generator_optimizer=g_optimizer,
        discriminator_optimizer=d_optimizer,
        epochs=epochs,
        generator_scheduler=None,
        discriminator_scheduler=None,
        train_dataloader=train_dataloader,
        validation_dataloader=val_dataloader,
        transformer_path=bert_save_path,
        discriminator_path=discriminator_save_path,
        generator_path=generator_save_path,
    )

    # Test the model
    test_acc, test_f1 = test_gan_bert(
        bert_model_name=model_name,
        transformer_path=bert_save_path,
        generator_path=generator_save_path,
        discriminator_path=discriminator_save_path,
        bow_mode=False,
        test_dataloader=test_dataloader
    )
    g1_bert_acc_list.append(test_acc)
    g1_bert_f1_list.append(test_f1)

    title_suffix = " " + str(int(100*(1-unsupervised_ratio))) + f"% supervised"
    # Visualize results
    plot_results_gan(gan1_training_results, title_suffix, plot_save_path)

np.save(file="../Plots/generator1/acc.npy", arr=np.array(g1_bert_acc_list))
np.save(file="../Plots/generator1/f1.npy", arr=np.array(g1_bert_f1_list))

## G2

In [ ]:
torch.manual_seed(42)
train_batch_size = 4
val_test_batch_size = 4
epochs = 10
learning_rate_discriminator = 5e-5
learning_rate_generator = 5e-5
max_size = 64
model_name = "bert-base-cased"
bert_tokenizer, bert_config = get_tokenizer(model_name=model_name)

In [ ]:
if not os.path.exists("../Plots/generator2/"):
    os.mkdir(path="../Plots/generator2/")

In [ ]:
g2_bert_acc_list = []
g2_bert_f1_list = []
for unsupervised_ratio in [0.99, 0.95, 0.90, 0.50]:
    bert_save_path = (
        f"../Plots/generator2/bert_{int(100*(1-unsupervised_ratio))}sup.pth"
    )
    discriminator_save_path = (
        f"../Plots/generator2/discriminator_{int(100*(1-unsupervised_ratio))}sup.pth"
    )
    generator_save_path = (
        f"../Plots/generator2/generator_{int(100*(1-unsupervised_ratio))}sup.pth"
    )
    plot_save_path = f"../Plots/generator2/"

    bert_model, _ = get_bert_model(model_name=model_name)
    classifier = Discriminator()
    generator, _ = get_bert_model(model_name=model_name)

    # Get dataloaders
    train_dataloader, val_dataloader, test_dataloader = generate_dataloader(
        dataset_folder_path=dataset_path,
        unsupervised_ratio=unsupervised_ratio,
        tokenizer=bert_tokenizer,
        train_batch_size=train_batch_size,
        valid_batch_size=val_test_batch_size,
        test_batch_size=val_test_batch_size,
        max_length=max_size,
        use_unsup=True,
        use_bow_dataset=True,
        random_seed=42,
    )

    # Define optimizers
    discriminator_params = [v for v in bert_model.parameters()] + [
        v for v in classifier.parameters()
    ]
    generator_params = [v for v in generator.parameters()]
    d_optimizer = torch.optim.AdamW(
        discriminator_params, lr=learning_rate_discriminator
    )
    g_optimizer = torch.optim.AdamW(generator_params, lr=learning_rate_generator)

    bert_model, generator, classifier, gan2_training_results = train_gan(
        transformer=bert_model,
        generator=generator,
        discriminator=classifier,
        bow_mode=True,
        generator_optimizer=g_optimizer,
        discriminator_optimizer=d_optimizer,
        epochs=epochs,
        generator_scheduler=None,
        discriminator_scheduler=None,
        train_dataloader=train_dataloader,
        validation_dataloader=val_dataloader,
        transformer_path=bert_save_path,
        discriminator_path=discriminator_save_path,
        generator_path=generator_save_path,
    )

    # Test the model
    test_acc, test_f1 = test_gan_bert(
        bert_model_name=model_name,
        transformer_path=bert_save_path,
        generator_path=generator_save_path,
        discriminator_path=discriminator_save_path,
        bow_mode=True,
        test_dataloader=test_dataloader
    )
    g2_bert_acc_list.append(test_acc)
    g2_bert_f1_list.append(test_f1)

    title_suffix = " " + str(int(100*(1-unsupervised_ratio))) + f"% supervised"
    # Visualize results
    plot_results_gan(gan2_training_results, title_suffix, plot_save_path)

np.save(file="../Plots/generator2/acc.npy", arr=np.array(g2_bert_acc_list))
np.save(file="../Plots/generator2/f1.npy", arr=np.array(g2_bert_f1_list))